In [1]:
import numpy as np

### Common Emitter Analysis

In [61]:
class CE_AMP_calculator:
    def __init__(self, Vcc, Ic, Vbe, B, fmin, Rs):
        self.Vcc = Vcc
        self.Ic = Ic
        self.Vbe = Vbe
        self.B = B
        self.fmin = fmin
        self.Rs = Rs

        self.Ve = 0.1 * Vcc
        # Load point is selected on center point of load line.
        self.Vce = Vcc / 2


    def _calculate_circuit_resistors(self):
        # 1. Resistors (based on design targets)
        self.Re = self.Ve / self.Ic
        self.Rc = (self.Vcc - self.Vce - self.Ve) / self.Ic

        # 2. Bias Resistors (10:1 divider rule)
        Vb = self.Ve + self.Vbe
        Ib = self.Ic / self.B
        Idiv = 10 * Ib

        self.R2 = Vb / Idiv
        self.R1 = (self.Vcc - Vb) / Idiv

        return {'Re':self.Re, 'Rc': self.Rc, 'R1': self.R1, 'R2': self.R2}

    def _calculate_AC_parameters(self, Rl, Ie_actual=None): #simulatd Ie
        if not hasattr(self, 'R1'):
            self._calculate_circuit_resistors()

        Ie = Ie_actual if Ie_actual is not None else self.Ic

        # --- AC Small-Signal Parameters ---
        # Dynamic emitter resistance
        re_ac = 25e-3 / Ie


        # Thevenin Resistance of bias network
        Rth = (1/self.R1 + 1/self.R2)**-1

        # --- C_E Calculation ---
        # Resistance looking into the base reflected to the emitter:
        # R_reflected = (Rth || Rs) / B
        if self.Rs == 0:
            R_parallel_base = Rth # Rth || infinity is Rth (avoid division by zero if R_S = 0)
        else:
            R_parallel_base = (1/Rth + 1/self.Rs)**-1

        # R'_E = R_E || (re_ac + R_reflected)
        R_reflected_to_emitter = re_ac + R_parallel_base / self.B
        Re_ac = (1 / self.Re + 1 / R_reflected_to_emitter)**-1
        # Capacitor formula for f_L = 1 / (2*pi*R*C)
        Ce = 1 / (2 * np.pi * self.fmin * Re_ac)

        # --- C_in Calculation ---
        # Amplifier Input Impedance: Zin = Rth || Zin_base
        Zin_base = self.B * re_ac
        Zin = (1/Zin_base + 1/Rth)**-1

        Rin_total = Zin + self.Rs
        Cin = 1 / (2*np.pi*self.fmin*Rin_total)

        # --- C_out Calculation ---
        # Amplifier Output Impedance: Zout = Rc
        Zout = self.Rc
        Rout_total = Zout + Rl
        Cout = 1 / (2*np.pi*self.fmin*Rout_total)

        Av = -self.Rc / re_ac
        return {'Ce': Ce, 'Cin': Cin, 'Cout': Cout, 'Av': Av}

    def _emitter_degeneration_calculations(self, Av_new, Rl, Ie_actual=None):

        if not hasattr(self, 'R1'):
            self._calculate_circuit_resistors()

        Ie = Ie_actual if Ie_actual is not None else self.Ic
        re_ac = 25e-3 / Ie
        Rth = (1/self.R1 + 1/self.R2)**-1

        Re1 = -self.Rc / np.abs(Av_new) - re_ac
        Re2 = self.Re - Re1

        # --- C_in Calculation ---
        # Amplifier Input Impedance: Zin = Rth || Zin_base
        Zin_base = self.B * (re_ac + Re1)
        Zin = (1/Zin_base + 1/Rth)**-1

        Rin_total = Zin + self.Rs
        Cin = 1 / (2*np.pi*self.fmin*Rin_total)

        # --- C_out Calculation ---
        # Amplifier Output Impedance: Zout = Rc
        Zout = self.Rc
        Rout_total = Zout + Rl
        Cout = 1 / (2*np.pi*self.fmin*Rout_total)

        # --- C_E Calculation ---
        # R_reflected = (Rth || Rs) / B
        if self.Rs == 0:
            R_parallel_base = Rth # Rth || infinity is Rth (avoid division by zero if R_S = 0)
        else:
            R_parallel_base = (1/Rth + 1/self.Rs)**-1

        R_CE_total = (1/Re2 + 1/(re_ac + Re1 + R_parallel_base/self.B))**-1
        Ce = 1 / (2 * np.pi * self.fmin * R_CE_total)
        return {'Cin': Cin, 'Cout': Cout, 'Ce': Ce}

In [62]:
calc = CE_AMP_calculator(
    Vcc=12,
    Ic=1e-3,
    Vbe=0.7,
    B=300,
    fmin=20,
    Rs=50
)

In [63]:
resistors = calc._calculate_circuit_resistors()
for k, v in resistors.items():
    print(f'{k}: {round(v/1e3, 2)}kΩ')

Re: 1.2kΩ
Rc: 4.8kΩ
R1: 303.0kΩ
R2: 57.0kΩ


In [64]:
capcitors = calc._calculate_AC_parameters(
    Rl = 1e4
)
for k, v in capcitors.items():
    if k == 'Av':
        print(f'{k}: {v}')
    else:
        print(f'{k}: {round(v/1e-6, 2)}uF')


Ce: 322.84uF
Cin: 1.22uF
Cout: 0.54uF
Av: -192.0


In [66]:
capcitors = calc. _emitter_degeneration_calculations(
    Av_new=-100,
    Rl = 1e4
)
for k, v in capcitors.items():
    if k == 'Av':
        print(f'{k}: {v}')
    else:
        print(f'{k}: {round(v/1e-6, 2)}uF')


Cin: -0.39uF
Cout: 0.54uF
Ce: -160.11uF
